In [1]:
from time import time
import numpy as np
numSeed = 42
np.random.seed(numSeed)
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Lambda, Conv2D, Conv2DTranspose, Activation, Flatten, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import cifar10

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.48
set_session(tf.Session(config=config))

from util import plotScatterDecode, plotProgress, plotCompDecode, plotScatterEncode, addNoise

Using TensorFlow backend.


## Declare the parameters

In [2]:
numEpochs = 200
sizeBatch = 32
sizeKernel = 3
dimInter1 = 1024
dimInter2 = 512
dimInter3 = 256
dimEncode = 128
layer_filters = [8, 32, 128, 512]
stdEps = 1.0 
ratRecon = 0.998
factNoise = 0
nameOptim = 'adam'

## Load the datasets

In [3]:
(xTrain, _), (xTest, yTest) = cifar10.load_data()
yTest = np.squeeze(yTest, axis=1)
sizeDigit = xTrain.shape[1]
numTrain = len(xTrain)
numTest = len(xTest)
dimInput = xTrain.shape[1:]

xTrain = xTrain.astype('float32') / 255.
xTest = xTest.astype('float32') / 255.
xTrain

xTrain = np.reshape(xTrain, [-1, *dimInput])
xTest = np.reshape(xTest, [-1, *dimInput])
xTrainNoise = addNoise(xTrain, factNoise=factNoise)
xTestNoise = addNoise(xTest, factNoise=factNoise)
print(xTrain.shape)
print(xTest.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


### Construct the encoder and decoder

In [4]:
inputs = Input(shape=(dimInput))  # adapt this if using `channels_first` image data format
x = inputs
# Stack of Conv2D blocks
# Notes:
# 1) Use Batch Normalization before ReLU on deep networks
# 2) Use MaxPooling2D as alternative to strides>1
# - faster but not as good as strides>1
for filters in layer_filters:
    x = Conv2D(filters=filters,
               kernel_size=sizeKernel,
               strides=2,
               activation='relu',
               padding='same')(x)

# Shape info needed to build Decoder Model
shape = K.int_shape(x)

# Generate the latent vector
x = Flatten()(x)
x = Dense(dimInter1, activation='relu')(x)
x = Dense(dimInter2, activation='relu')(x)
x = Dense(dimInter3, activation='relu')(x)
zMean = Dense(dimEncode)(x)
zSigmaLog = Dense(dimEncode)(x) # log for linear dense

def sampling(args):
    zMean, zSigmaLog = args
    epsilon = K.random_normal(shape=(K.shape(zMean)[0], dimEncode),
                              mean=0., stddev=stdEps)
    return zMean + K.exp(zSigmaLog) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
# z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
z = Lambda(sampling)([zMean, zSigmaLog])
encoder = Model(inputs, z, name='encoder')
encoder.summary()

# Build the Decoder Model
inputLatent = Input(shape=(dimEncode,), name='decoder_input')
x = Dense(dimInter3, activation='relu')(inputLatent)
x = Dense(dimInter2, activation='relu')(x)
x = Dense(dimInter1, activation='relu')(x)
x = Dense(shape[1] * shape[2] * shape[3])(x)
x = Reshape((shape[1], shape[2], shape[3]))(x)

# Stack of Transposed Conv2D blocks
# Notes:
# 1) Use Batch Normalization before ReLU on deep networks
# 2) Use UpSampling2D as alternative to strides>1
# - faster but not as good as strides>1
for filters in layer_filters[::-1]:
    x = Conv2DTranspose(filters=filters,
                        kernel_size=sizeKernel,
                        strides=2,
                        activation='relu',
                        padding='same')(x)

x = Conv2DTranspose(filters=3,
                    kernel_size=sizeKernel,
                    padding='same')(x)

outputs = Activation('sigmoid', name='decoder_output')(x)

# Instantiate Decoder Model
decoder = Model(inputLatent, outputs, name='decoder')
decoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 8)    224         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 8, 8, 32)     2336        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 4, 4, 128)    36992       conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

## Construct the autoencoder

In [5]:
# Autoencoder = Encoder + Decoder
# Instantiate Autoencoder Model
autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
encoder (Model)              (None, 128)               3449984   
_________________________________________________________________
decoder (Model)              (None, 32, 32, 3)         5778307   
Total params: 9,228,291
Trainable params: 9,228,291
Non-trainable params: 0
_________________________________________________________________


## Compile and train

In [ ]:
def lossVAE(zMean, zSigmaLog):
    def loss(tensorInput, tensorDecode):
        lossRecon =  metrics.binary_crossentropy(K.flatten(tensorInput), K.flatten(tensorDecode))
        lossKL = - 0.5 * K.sum(1 + 2 * zSigmaLog - K.square(zMean) - K.square(K.exp(zSigmaLog)), axis=-1)
#         lossKL = - 0.5 * K.mean(1 + zSigmaLog - K.square(zMean) - K.exp(zSigmaLog), axis=-1)
        return ratRecon * lossRecon + (1 - ratRecon) * lossKL
    return loss

autoencoder.compile(optimizer=nameOptim, loss=lossVAE(zMean, zSigmaLog))

# Train the autoencoder
tic = time()
history = autoencoder.fit(xTrainNoise, xTrain,
                epochs=numEpochs,
                batch_size=sizeBatch,
                shuffle=True,
                validation_data=(xTest, xTest))
timeTrain = time() - tic

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 22s 434us/step - loss: 0.6612 - val_loss: 0.6507
Epoch 2/200
50000/50000 [==============================] - 20s 392us/step - loss: 0.6446 - val_loss: 0.6436
Epoch 3/200
50000/50000 [==============================] - 19s 390us/step - loss: 0.6431 - val_loss: 0.6435
Epoch 4/200
50000/50000 [==============================] - 20s 403us/step - loss: 0.6424 - val_loss: 0.6417
Epoch 5/200
50000/50000 [==============================] - 21s 420us/step - loss: 0.6396 - val_loss: 0.6398
Epoch 6/200
50000/50000 [==============================] - 22s 440us/step - loss: 0.6384 - val_loss: 0.6380
Epoch 7/200
50000/50000 [==============================] - 22s 447us/step - loss: 0.6370 - val_loss: 0.6372
Epoch 8/200
50000/50000 [==============================] - 21s 419us/step - loss: 0.6367 - val_loss: 0.6371
Epoch 9/200
50000/50000 [==============================] - 21s 422us/step - loss: 0.63

Epoch 76/200
50000/50000 [==============================] - 29s 581us/step - loss: 0.6306 - val_loss: 0.6318
Epoch 77/200
50000/50000 [==============================] - 30s 600us/step - loss: 0.6306 - val_loss: 0.6315
Epoch 78/200
50000/50000 [==============================] - 29s 590us/step - loss: 0.6305 - val_loss: 0.6318
Epoch 79/200
50000/50000 [==============================] - 29s 580us/step - loss: 0.6305 - val_loss: 0.6321
Epoch 80/200
50000/50000 [==============================] - 27s 543us/step - loss: 0.6304 - val_loss: 0.6326
Epoch 81/200
50000/50000 [==============================] - 27s 544us/step - loss: 0.6306 - val_loss: 0.6315
Epoch 82/200
50000/50000 [==============================] - 28s 551us/step - loss: 0.6304 - val_loss: 0.6317
Epoch 83/200
50000/50000 [==============================] - 28s 565us/step - loss: 0.6304 - val_loss: 0.6318
Epoch 84/200
50000/50000 [==============================] - 28s 557us/step - loss: 0.6304 - val_loss: 0.6317
Epoch 85/200
50000/

50000/50000 [==============================] - 23s 467us/step - loss: 0.6297 - val_loss: 0.6315
Epoch 151/200
50000/50000 [==============================] - 23s 463us/step - loss: 0.6297 - val_loss: 0.6313
Epoch 152/200
50000/50000 [==============================] - 23s 461us/step - loss: 0.6297 - val_loss: 0.6317
Epoch 153/200
50000/50000 [==============================] - 23s 463us/step - loss: 0.6296 - val_loss: 0.6319
Epoch 154/200
50000/50000 [==============================] - 23s 457us/step - loss: 0.6300 - val_loss: 0.6313
Epoch 155/200
50000/50000 [==============================] - 23s 461us/step - loss: 0.6296 - val_loss: 0.6313
Epoch 156/200
50000/50000 [==============================] - 23s 459us/step - loss: 0.6296 - val_loss: 0.6329
Epoch 157/200
50000/50000 [==============================] - 23s 459us/step - loss: 0.6297 - val_loss: 0.6313
Epoch 158/200
50000/50000 [==============================] - 23s 467us/step - loss: 0.6295 - val_loss: 0.6319
Epoch 159/200
50000/5000

## Show the historical training progress 

In [ ]:
# print("traing consumed: " + str(timeTrain) + " seconds")
plotProgress(history)

## Get the encoding and decoding results of testing data, and get the mean/std of the encoding 

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
encodeTest = encoder.predict(xTestNoise)
decodeTest = decoder.predict(encodeTest)
meanEncTest = np.mean(encodeTest, axis=0)
stdEncTest = np.std(encodeTest, axis=0)

## Compare original digitals with the decoding results

In [ ]:
plotCompDecode(xTest, decodeTest, sizeDigit = (32, 32, 3))

## Plot the scatter of the encoding space

In [ ]:
xlim = (meanEncTest[0] - 4*stdEncTest[0], meanEncTest[0] + 4*stdEncTest[0])
ylim = (meanEncTest[1] - 4*stdEncTest[1], meanEncTest[1] + 4*stdEncTest[1])

plotScatterEncode(encodeTest, yTest, xlim, ylim, numShow=10000)
scoreSilh = silhouette_score(encodeTest, yTest)  

## Plot the decoding results from the encoding scatter 

In [ ]:
# display a 2D manifold of the digits
plotScatterDecode(decoder, (32,32,3), xlim, ylim)

In [ ]:
print(timeTrain, history.history["loss"][numEpochs-1], history.history["val_loss"][numEpochs-1], scoreSilh)

In [ ]:
(xTrain, _), (xTest, yTest) = mnist.load_data()

In [ ]:
xTest.shape

In [ ]:
dimInput

In [ ]:
print(*dimInput)